<a href="https://colab.research.google.com/github/helvete74/IFAGE_dev_AI/blob/main/Efficient_Object_Detection_with_YOLOV8_and_KerasCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TUTO Efficient Object Detection with YOLOV8 and KerasCV

https://keras.io/examples/vision/yolov8/

In [1]:
!pip install roboflow

In [20]:
# download the dataset from roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4RTem5JDv2kRxw8jk1El")

# version yoloV8
# project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
# version = project.version(3)
# dataset = version.download("yolov8")

#Pascal VOC XML
project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
version = project.version(3)
dataset = version.download("voc")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Car-3 in voc:: 100%|██████████| 59603/59603 [00:20<00:00, 2961.70it/s]


install KerasCV

In [3]:
!pip install --upgrade git+https://github.com/keras-team/keras-cv -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Import lib

In [4]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization


Using TensorFlow backend


Hyperparameters

In [5]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0


A dictionary is created to map each class name to a unique numerical identifier. This mapping is used to encode and decode the class labels during training and inference in object detection tasks.

In [26]:
'''
class_ids = [
    "car",
    "pedestrian",
    "trafficLight",
    "biker",
    "truck",
]
'''
# class_ids from the Udacity Self Driving Car Dataset
class_ids = [
    "car",
    "pedestrian",
    "trafficLight-Red",
    "trafficLight-Green",
    "truck",
    "trafficLight",
    "biker",
    "trafficLight-RedLeft",
    "trafficLight-GreenLeft",
    "trafficLight-Yellow",
    "trafficLight-YellowLeft"
]



class_mapping = dict(zip(range(len(class_ids)), class_ids))
class_mapping

{0: 'car',
 1: 'pedestrian',
 2: 'trafficLight-Red',
 3: 'trafficLight-Green',
 4: 'truck',
 5: 'trafficLight',
 6: 'biker',
 7: 'trafficLight-RedLeft',
 8: 'trafficLight-GreenLeft',
 9: 'trafficLight-Yellow',
 10: 'trafficLight-YellowLeft'}

In [27]:
# Path to images and annotations
# path_images = "/kaggle/input/dataset/data/images/"
# path_annot = "/kaggle/input/dataset/data/annotations/"
# local path :
path_annot = "Self-Driving-Car-3/export"
path_images  = "Self-Driving-Car-3/export"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)

In [23]:
jpg_files

['Self-Driving-Car-3/export/1478019952686311006_jpg.rf.54e2d12dbabc46be3c78995b6eaf3fee.jpg',
 'Self-Driving-Car-3/export/1478019952686311006_jpg.rf.JLSB3LP2Q4RuGHYKqfF6.jpg',
 'Self-Driving-Car-3/export/1478019953180167674_jpg.rf.8a816c9d7e9b423a63ed6ecd4a663e47.jpg',
 'Self-Driving-Car-3/export/1478019953180167674_jpg.rf.azslsZnM8FLQPu3QWLTl.jpg',
 'Self-Driving-Car-3/export/1478019953689774621_jpg.rf.2e4b7ae29c3379da1282e85cff4c1745.jpg',
 'Self-Driving-Car-3/export/1478019953689774621_jpg.rf.UpOkvtBppZZaLfvVZ0JX.jpg',
 'Self-Driving-Car-3/export/1478019954186238236_jpg.rf.cbpP963e2sN7rHIZMJtD.jpg',
 'Self-Driving-Car-3/export/1478019954186238236_jpg.rf.f8ac6ad31e400d4918ff394f903bb95c.jpg',
 'Self-Driving-Car-3/export/1478019954685370994_jpg.rf.049fda8ca86adf1e404b41955eb4aba8.jpg',
 'Self-Driving-Car-3/export/1478019954685370994_jpg.rf.Jah8pEPis5n1x6iQCtmT.jpg',
 'Self-Driving-Car-3/export/1478019955185244088_jpg.rf.gJpj2eCO1Dd7Sic9WlhE.jpg',
 'Self-Driving-Car-3/export/1478019955

In [24]:
xml_files

['Self-Driving-Car-3/export/1478019952686311006_jpg.rf.54e2d12dbabc46be3c78995b6eaf3fee.xml',
 'Self-Driving-Car-3/export/1478019952686311006_jpg.rf.JLSB3LP2Q4RuGHYKqfF6.xml',
 'Self-Driving-Car-3/export/1478019953180167674_jpg.rf.8a816c9d7e9b423a63ed6ecd4a663e47.xml',
 'Self-Driving-Car-3/export/1478019953180167674_jpg.rf.azslsZnM8FLQPu3QWLTl.xml',
 'Self-Driving-Car-3/export/1478019953689774621_jpg.rf.2e4b7ae29c3379da1282e85cff4c1745.xml',
 'Self-Driving-Car-3/export/1478019953689774621_jpg.rf.UpOkvtBppZZaLfvVZ0JX.xml',
 'Self-Driving-Car-3/export/1478019954186238236_jpg.rf.cbpP963e2sN7rHIZMJtD.xml',
 'Self-Driving-Car-3/export/1478019954186238236_jpg.rf.f8ac6ad31e400d4918ff394f903bb95c.xml',
 'Self-Driving-Car-3/export/1478019954685370994_jpg.rf.049fda8ca86adf1e404b41955eb4aba8.xml',
 'Self-Driving-Car-3/export/1478019954685370994_jpg.rf.Jah8pEPis5n1x6iQCtmT.xml',
 'Self-Driving-Car-3/export/1478019955185244088_jpg.rf.gJpj2eCO1Dd7Sic9WlhE.xml',
 'Self-Driving-Car-3/export/1478019955

In [28]:
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_images, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids


image_paths = []
bbox = []
classes = []
for xml_file in tqdm(xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)


  0%|          | 0/29800 [00:00<?, ?it/s]

In [30]:
classes

[[4, 0, 0, 1],
 [4, 0, 0, 1],
 [4, 0, 4, 0],
 [4, 0, 4, 0],
 [1, 4, 0, 0, 0, 0],
 [1, 4, 0, 0, 0, 0],
 [1, 4, 0, 0, 0, 0],
 [1, 4, 0, 0, 0, 0],
 [1, 4, 0, 0, 0],
 [1, 4, 0, 0, 0],
 [1, 4, 0, 0, 0],
 [4, 1, 0, 4, 0],
 [4, 1, 0, 4, 0],
 [0, 0, 1, 0, 4, 0],
 [0, 0, 1, 0, 4, 0],
 [0, 0, 0, 0, 1, 0, 4, 0],
 [0, 0, 0, 0, 1, 0, 4, 0],
 [0, 0, 0, 0, 1, 0, 4, 0, 0],
 [0, 0, 0, 0, 1, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0],
 [0, 0, 1, 4, 0, 0],
 [0, 0, 1, 4, 0, 0],
 [0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0,

In [31]:
# create irregular tensor with tf.ragged
bbox = tf.ragged.constant(bbox)
classes = tf.ragged.constant(classes)
image_paths = tf.ragged.constant(image_paths)

# create the dataset
data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))


Splitting data in training and validation data

In [32]:
# Determine the number of validation samples
num_val = int(len(xml_files) * SPLIT_RATIO)

# Split the dataset into train and validation sets
val_data = data.take(num_val)
train_data = data.skip(num_val)




```
bounding_boxes = {
    # num_boxes may be a Ragged dimension
    'boxes': Tensor(shape=[batch, num_boxes, 4]),
    'classes': tf.Tensor(shape=[batch, num_boxes])
}
```




In [35]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}